In [1]:
library(disk.frame)

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: purrr

Registered S3 method overwritten by 'pryr':
  method      from
  print.bytes Rcpp


## Message from disk.frame:
We have 1 workers to use with disk.frame.
To change that, use setup_disk.frame(workers = n) or just setup_disk.frame() to use the defaults.


It is recommended that you run the following immediately to set up disk.frame with multiple workers in order to parallelize your operations:


```r
# this will set up disk.frame with multiple workers
setup_disk.frame()
# this will allow unlimited amount of data to be passed from worker to worker
options(future.globals.maxSize = Inf)
```





Attaching package: ‘disk.frame’


The following objects are masked from ‘package:purrr’:

    imap, imap_dfr, map, map2


The following

In [2]:
setup_disk.frame()

The number of workers available for disk.frame is 6



In [6]:
system.time(a <- csv_to_disk.frame("../fm-data/perf/Performance_2000Q1.txt", header=FALSE))

   user  system elapsed 
 25.838   1.435   7.742 

In [7]:
head(a)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31
<int64>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
100007365142,01/01/2000,NULL,8,NA,0,360,359,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NA,NULL,NA,NULL
100007365142,01/01/2001,NULL,8,74319.00,12,348,347,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NA,NULL,NA,NULL
100007365142,01/01/2002,NULL,8,73635.48,24,336,335,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NA,NULL,NA,NULL
100007365142,01/01/2003,NULL,8,72795.41,36,324,322,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NA,NULL,NA,NULL
100007365142,02/01/2000,NULL,8,NA,1,359,358,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NA,NULL,NA,NULL
100007365142,02/01/2001,NULL,8,74264.14,13,347,346,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NA,NULL,NA,NULL


In [16]:
system.time(b <- a %>%
  srckeep("V6") %>%
  summarise(mean(V6)) %>%
  collect)

b

   user  system elapsed 
  0.274   0.004   1.650 

mean(V6)
<dbl>
33.96503


In [23]:
files = dir("../fm-data/perf", full.names=T)
file.size(files)/1024^3

[1] 0.9272777 0.8341817 0.8858110 1.0335631

In [24]:
system.time(a <- csv_to_disk.frame(files, header=FALSE))

csv_to_disk.frame: Reading multiple input files.

Please use `colClasses = `  to set column types to minimize the chance of a failed read




 ----------------------------------------------------- 

-- Converting CSVs to disk.frame -- Stage 1 of 2:



Converting 4 CSVs to 6 disk.frames each consisting of 6 chunks





 Progress: ──────────────────────────────────────────────────────────────── 100%



-- Converting CSVs to disk.frame -- Stage 1 or 2 took: 19.7s elapsed (0.248s cpu)

 ----------------------------------------------------- 

 

 ----------------------------------------------------- 

-- Converting CSVs to disk.frame -- Stage 2 of 2:



Row-binding the 6 disk.frames together to form one large disk.frame:

Creating the disk.frame at /tmp/Rtmpk244dn/file153443d37d1.df



Appending disk.frames: 



 Progress: ──────────────────────────────────────────────────────────────── 100%



Stage 2 of 2 took: 14.1s elapsed (0.405s cpu)

 ----------------------------------------------------- 

Stage 1 & 2 in total took: 33.8s elapsed (0.685s cpu)



   user  system elapsed 
  0.693   0.051  33.840 

In [25]:
nrow(a)

[1] 36190382

In [27]:
library(fst)

In [28]:
file.size(a)

ERROR: Error in file.info(..., extra_cols = FALSE): invalid filename argument


In [35]:
sum(file.size(dir(attr(a, "path"), full.names=T)))/1024^3

[1] 1.275912

In [37]:
head(a)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31
<int64>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
100007365142,01/01/2000,NULL,8,NA,0,360,359,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NA,NULL,NA,NULL
100007365142,01/01/2001,NULL,8,74319.00,12,348,347,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NA,NULL,NA,NULL
100007365142,01/01/2002,NULL,8,73635.48,24,336,335,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NA,NULL,NA,NULL
100007365142,01/01/2003,NULL,8,72795.41,36,324,322,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NA,NULL,NA,NULL
100007365142,02/01/2000,NULL,8,NA,1,359,358,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NA,NULL,NA,NULL
100007365142,02/01/2001,NULL,8,74264.14,13,347,346,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NA,NULL,NA,NULL


In [41]:
system.time(a1 <- a %>%
  get_chunk(1) %>%
  mutate(as.Date(paste0("01", V1), "%d%m/%Y")))

   user  system elapsed 
  5.994   1.781   5.221 

In [40]:
a1

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V23,V24,V25,V26,V27,V28,V29,V30,V31,"as.Date(paste0(""01"", V1), ""%d%m/%Y"")"
<int64>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<date>
100007365142,01/01/2000,NULL,8,NA,0,360,359,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NULL,NA,NULL,NA
100007365142,01/01/2001,NULL,8,74319.00,12,348,347,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NULL,NA,NULL,NA
100007365142,01/01/2002,NULL,8,73635.48,24,336,335,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NULL,NA,NULL,NA
100007365142,01/01/2003,NULL,8,72795.41,36,324,322,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NULL,NA,NULL,NA
100007365142,02/01/2000,NULL,8,NA,1,359,358,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NULL,NA,NULL,NA
100007365142,02/01/2001,NULL,8,74264.14,13,347,346,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NULL,NA,NULL,NA
100007365142,02/01/2002,NULL,8,73576.06,25,335,334,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NULL,NA,NULL,NA
100007365142,02/01/2003,NULL,8,72680.39,37,323,320,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NULL,NA,NULL,NA
100007365142,03/01/2000,NULL,8,NA,2,358,357,01/2030,0,⋯,NA,NA,NA,NA,NA,NA,NULL,NA,NULL,NA
